In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os
from sklearn.svm import SVC

model = VGG16(weights='imagenet', include_top=False)
#model.summary()



def extract_features(path):
    
    feature_advert_list = []


    for img_path in os.listdir(path):
        if(img_path != '.DS_Store'):
            img = image.load_img(path+'/'+img_path, target_size=(161,81))
            #img = image.load_img(path+'/'+img_path, target_size=(360, 64))
            img_data = image.img_to_array(img)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)

            vgg16_feature = model.predict(img_data)
            vgg16_feature_np = np.array(vgg16_feature)

            feature_advert_list.append(vgg16_feature_np.flatten())

     
    return np.array(feature_advert_list)

def extract_features_image(img_path):
    
    feature_list = []


    #for img_path in os.listdir(path):
    
    img = image.load_img(img_path, target_size=(161,81))
    #img = image.load_img(img_path, target_size=(360, 64))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    vgg16_feature = model.predict(img_data)
    vgg16_feature_np = np.array(vgg16_feature)

    feature_list.append(vgg16_feature_np.flatten())

     
    return np.array(feature_list)



In [ ]:
path_bowler_action = '/Users/bhavanirishitharavipati/Desktop/Machine Learning/Replay'
path_non_bowler = '/Users/bhavanirishitharavipati/Desktop/Machine Learning/Non_Replay'
features_list_bowler_action = extract_features(path_bowler_action)
features_list_non_bowler = extract_features(path_non_bowler)

labels_non_bowler = np.ones(len(features_list_non_bowler))
labels_bowler_action = np.zeros(len(features_list_bowler_action))
feat_bowler_action = np.column_stack((features_list_bowler_action,labels_bowler_action))
feat_non_bowler = np.column_stack((features_list_non_bowler,labels_non_bowler))

feat_x_y = np.concatenate((feat_bowler_action,feat_non_bowler),axis=0)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

#print(len(feat_x_y[0]))
Y_data = feat_x_y[:,5120]
#print(Y_data)
X_data = feat_x_y[:,0:5120]

x_tr,x_ts,y_tr,y_ts = train_test_split(X_data, Y_data, test_size=0.2,random_state=10)
clf = LinearSVC(C=10)
#print(len(x_tr),len(y_tr))
clf.fit(x_tr,y_tr)
predictions_tr = (clf.predict(x_ts))
scores = cross_val_score(clf, x_tr, y_tr, cv=10)
test_acc = accuracy_score(y_ts,predictions_tr)

# print("Training Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# print("Test Accuracy: %0.4f" % test_acc)

In [ ]:
import cv2
from PIL import Image

def crop_img(img):
    center_x, center_y = img.shape[1] / 2, img.shape[0] / 2
    height = center_y*2
    height_scaled = img.shape[0] * 0.80
    width = center_x*2
    width_scaled = img.shape[1] * 0.74
    top_y, bottom_y = height_scaled , height 
    left_x, right_x = 0, width-width_scaled
    img_cropped = img[int(top_y):int(bottom_y), int(left_x):int(right_x)]
    return img_cropped
    
    
def FrameCapture(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
  
    # Used as counter variable 
    count = 0
  
    # checks whether frames were extracted 
    success = 1
    
  
    while success: 
  
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read() 
  
        # Saves the frames with frame-count 
        if(success):
            cv2.imwrite("/Users/bhavanirishitharavipati/Desktop/umpire_new_2/%d.jpg" % count, image)

            im = cv2.imread("/Users/bhavanirishitharavipati/Desktop/umpire_new_2/"+str(count)+".jpg")
            
            img_cropped = crop_img(im)
        
            cv2.imwrite("/Users/bhavanirishitharavipati/Desktop/Machine Learning/cropped_image/%d.jpg" % count, img_cropped)
            
            feature = extract_features_image("/Users/bhavanirishitharavipati/Desktop/Machine Learning/cropped_image/"+str(count)+".jpg")
        
           
            if(clf.predict(feature) == 1):
                    os.remove("/Users/bhavanirishitharavipati/Desktop/umpire_new_2/"+str(count)+".jpg")
            
            count += 1
        

            

In [ ]:
#FrameCapture('/Users/bhavanirishitharavipati/Desktop/sample2_clip.mp4')